# 資料前處理(Missing data, One-hot encoding, Feature Scaling)
資料的品質、特徵的選取決定了機器學習的上限，模型(Model)只是逼近這個上限。<br>
雖然在學術界總是以Model為主要討論對象，但實際上在業界80%的時間都是在對資料進行前處理，<br>
包含了資料獲取、清理、特徵選擇、特徵處理…到這裡我們可以稍微了解資料前處理的重要性。<br>

常見的資料前處理如下所示：<br>
1.缺失值的處理<br>
2.類別資料的處理（有序、無序）<br>
3.資料特徵縮放<br>

In [4]:
import numpy as np
import pandas as pd
from sklearn import datasets
from io import StringIO
import math
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. 缺失值的處理
缺值主要處理方式有兩種：<br>
*丟棄，如果資料量夠多<br>
*補值<br>

In [6]:
csv_data = '''A,B,C,D,E
            5.0,2.0,3.0,,6
            1.0,6.0,,8.0,5
            0.0,11.0,12.0,4.0,5
            3.0,,3.0,5.0,
            5.0,1.0,4.0,2.0,4
           '''
df = pd.read_csv(StringIO(csv_data))

In [7]:
df = pd.read_csv('/content/drive/MyDrive/IMLP375/Unit03/preText.csv')

In [ ]:
df

,A,B,C,D,E
0,5.0,2.0,3.0,NaN,6.0
1,1.0,6.0,NaN,8.0,5.0
2,0.0,11.0,12.0,4.0,5.0
3,3.0,NaN,3.0,5.0,NaN
4,5.0,1.0,4.0,2.0,4.0


## Missing Data(空值資料處理)

使用Pandas去空值的方法“dropna”，dropna預設只要任一欄位有空值，就會整筆刪掉。<br>
可透過參數來調整，像是把how設為all，就是要全部為空才清掉，或是用subset指定當某一欄為空時才刪。

In [8]:
df.dropna()

,A,B,C,D,E
2,0.0,11.0,12.0,4.0,5.0
4,5.0,1.0,4.0,2.0,4.0


In [9]:
df.dropna(how = 'all')

,A,B,C,D,E
0,5.0,2.0,3.0,NaN,6.0
1,1.0,6.0,NaN,8.0,5.0
2,0.0,11.0,12.0,4.0,5.0
3,3.0,NaN,3.0,5.0,NaN
4,5.0,1.0,4.0,2.0,4.0


In [10]:
df.dropna(subset = ['C'])

,A,B,C,D,E
0,5.0,2.0,3.0,NaN,6.0
2,0.0,11.0,12.0,4.0,5.0
3,3.0,NaN,3.0,5.0,NaN
4,5.0,1.0,4.0,2.0,4.0


補值則使用fillna函式即可，依照以下範例即可補上固定值0、平均數、眾數、中位數…

In [11]:
df.fillna(0)

,A,B,C,D,E
0,5.0,2.0,3.0,0.0,6.0
1,1.0,6.0,0.0,8.0,5.0
2,0.0,11.0,12.0,4.0,5.0
3,3.0,0.0,3.0,5.0,0.0
4,5.0,1.0,4.0,2.0,4.0


In [12]:
df['B'] = df['B'].fillna(df['B'].mean())
df

,A,B,C,D,E
0,5.0,2.0,3.0,NaN,6.0
1,1.0,6.0,NaN,8.0,5.0
2,0.0,11.0,12.0,4.0,5.0
3,3.0,5.0,3.0,5.0,NaN
4,5.0,1.0,4.0,2.0,4.0


In [13]:
df['C'] = df['C'].fillna(df['C'].mode()[0])
df

,A,B,C,D,E
0,5.0,2.0,3.0,NaN,6.0
1,1.0,6.0,3.0,8.0,5.0
2,0.0,11.0,12.0,4.0,5.0
3,3.0,5.0,3.0,5.0,NaN
4,5.0,1.0,4.0,2.0,4.0


In [14]:
df['D'] = df['D'].fillna(df['D'].median())
df

,A,B,C,D,E
0,5.0,2.0,3.0,4.5,6.0
1,1.0,6.0,3.0,8.0,5.0
2,0.0,11.0,12.0,4.0,5.0
3,3.0,5.0,3.0,5.0,NaN
4,5.0,1.0,4.0,2.0,4.0


In [15]:
df['E'] = df['E'].fillna(df['E'].min())
df

,A,B,C,D,E
0,5.0,2.0,3.0,4.5,6.0
1,1.0,6.0,3.0,8.0,5.0
2,0.0,11.0,12.0,4.0,5.0
3,3.0,5.0,3.0,5.0,4.0
4,5.0,1.0,4.0,2.0,4.0


## 2.Categorical Data(類別資料處理)
由於要在空間中表示點，所有的特徵都需要是數值，因此如果是類別的資料，<br>
像是XL,L,M,S,XS或是資料類別為Male, Female, Not Specified，就需要轉成數值才能在空間中來表示。<br>
有序的類別資料通常是直接使用數值替換，比方說XL, L, M, S, XS雖然是類別的屬性但因為有大小順序的關係，可以用10, 7 , 5, 3, 1來替換。<br>如果是Male, Female, Not Specified因為這三種都是等價的關係因此需要找一個方法讓這三個屬性距離原點是相同距離，<br>
One-hot encoding 就是解決這的問題的方法，首先會將Male, Female, Not Specified由Gender從成一個欄位拆成三個欄位，<br>
因此編號1的使用者的屬性資料就是(1,0,0)編號2的使用者就是(0,1,0) 編號三個使用者就是(0,0,1)這三個使用者對於原點的距離都是1，就達成我們想要的結果了。<br>
但One-hot encoding的方法只適合類別種類少的形況下，如果類別種類太多就會產生出一大堆的特徵，造成其他的問題（比方說維數災難）。

In [16]:
df2 = pd.DataFrame(
    [['green', 'M', 10.1, 1],
    ['red', 'L', 13.5, 2],
    ['blue', 'XL', 15.3, 1]]
)
df2.columns = ['color', 'size', 'price', 'classlabel']
df2

,color,size,price,classlabel
0,green,M,10.1,1
1,red,L,13.5,2
2,blue,XL,15.3,1


由於size是屬於有序的資料，我們只要稍微轉換為數值即可

In [17]:
size_mapping = {'XL':5, 'L':3, 'M':1}
df2['size'] = df2['size'].map(size_mapping)
df2

,color,size,price,classlabel
0,green,1,10.1,1
1,red,3,13.5,2
2,blue,5,15.3,1


至於color我們就用onehot-encoding的方法來處理，在pandas裡面要使用<br>
onehot-encoding使用get_dummies這個函式就可以了，範例如下


In [18]:
onehot_encoding = pd.get_dummies(df2['color'], prefix = 'color')
onehot_encoding

,color_blue,color_green,color_red
0,0,1,0
1,0,0,1
2,1,0,0


In [19]:
pd.concat([onehot_encoding, df2], axis = 1)

,color_blue,color_green,color_red,color,size,price,classlabel
0,0,1,0,green,1,10.1,1
1,0,0,1,red,3,13.5,2
2,1,0,0,blue,5,15.3,1


In [20]:
df2 = df2.drop('color', 1)

<ipython-input-20-c09386464986>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df2 = df2.drop('color', 1)


In [21]:
pd.concat([onehot_encoding, df2], axis = 1)

,color_blue,color_green,color_red,size,price,classlabel
0,0,1,0,1,10.1,1
1,0,0,1,3,13.5,2
2,1,0,0,5,15.3,1


# 3.資料特徵縮放
## 資料正規化(normalization)
最常見的Normalization為0–1區間縮放，經過Normalization之後資料的範圍會介在0~1之間，<br>
原本的最大值變為1，最小值變為0，具體作法如下圖


In [22]:
from IPython.display import Math

In [23]:
Math(r'x^{(i)}_{norm}=\frac{x^{(i)}-x_{min}}{x_{max}-x_{min}}')



<IPython.core.display.Math object>

In [24]:
import numpy as np
import pandas as pd
from sklearn import datasets
from io import StringIO

iris = datasets.load_iris()
x = pd.DataFrame(iris['data'], columns=iris['feature_names'])
print("target_names: "+str(iris['target_names']))
y = pd.DataFrame(iris['target'], columns=['target_names'])
data = pd.concat([x,y], axis=1)
data.head(3)

target_names: ['setosa' 'versicolor' 'virginica']


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target_names
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0


In [25]:
tmp = data['sepal length (cm)']
tmp = (tmp - tmp.min()) / (tmp.max() - tmp.min())
data['sepal length (cm)'] = tmp

tmp = data['sepal width (cm)']
tmp = (tmp - tmp.min()) / (tmp.max() - tmp.min())
data['sepal width (cm)'] = tmp

tmp = data['petal length (cm)']
tmp = (tmp - tmp.min()) / (tmp.max() - tmp.min())
data['petal length (cm)'] = tmp

tmp = data['petal width (cm)']
tmp = (tmp - tmp.min()) / (tmp.max() - tmp.min())
data['petal width (cm)'] = tmp

data.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target_names
0,0.222222,0.625000,0.067797,0.041667,0
1,0.166667,0.416667,0.067797,0.041667,0
2,0.111111,0.500000,0.050847,0.041667,0
3,0.083333,0.458333,0.084746,0.041667,0
4,0.194444,0.666667,0.067797,0.041667,0


## 資料標準化(Standardization)
經過Standardization資料的平均值會變為0, 標準差變為1

In [26]:
Math(r'x^{(i)}_{std}=\frac{x^{(i)}-\mu_{x}}{\sigma_{x}}')



<IPython.core.display.Math object>

In [27]:
tmp = data['sepal length (cm)']
tmp = (tmp - tmp.mean()) / tmp.std()
data['sepal length (cm)'] = tmp

data.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target_names
0,-0.897674,0.625000,0.067797,0.041667,0
1,-1.139200,0.416667,0.067797,0.041667,0
2,-1.380727,0.500000,0.050847,0.041667,0
3,-1.501490,0.458333,0.084746,0.041667,0
4,-1.018437,0.666667,0.067797,0.041667,0
